# Create Tornadoes

In [1]:
from pyincore import HazardService, IncoreClient
import json

In [2]:
client = IncoreClient()

Connection successful to IN-CORE services. pyIncore version detected: 0.9.4


In [3]:
hazardsvc = HazardService(client)

## Create Model Based Tornado

**Input:** Json representing a model based tornado. 

* tornadoModel (required) - specify tornado model (Mean width will create a tornado using the mean width from historical data for the EF rating)
    * Mean width tornado (MeanWidthTornado) - user specifies start/end points with a width equal to the average for that EF rating looking at historical tornado data
    * Random width tornado (RandomWidthTornado) - user specifies start/end points, random width selected use the historical data to determine the max width for that tornado rating
    * Mean Length/Width/angle (MeanLengthWidthAngleTornado) - user specifies start point, length/width and angle are based on the historical average for those attributes based on EF selected
    * Random length/width/angle (RandomLengthWidthAngleTornado) - user specifies start point, random length/width/angle selected for the ef rating using historical data
    * Random angle (RandomAngleTornado) - user specifies start point, random angle selected based on historical data.
* efRating (required) - specify the Enhanced Fujita (EF) scale intensity of the tornado (EF0 - EF5)
* startLatitude/startLongitude (required) - specify the starting location of the tornado
* endLatitude/endLongitude (optional)- depending on the model, specify an end latitude/longitude value. Some tornado models (e.g. mean length width, and angle) calcuate endLat and endLon, others (e.g. random angle) will generate multiple endpoints programmatically so the input must be passed as an array
* windSpeedMethod(optional) - for computing wind speed within an EF boundary, 0 indicates using linear interpolation, 1 indicates uniform random distribution. Default is Uniform random distribution.

In [4]:
with open("files/tornado-model.json", 'r') as file:
    tornado_model_json = file.read()
    print(json.dumps(json.loads(tornado_model_json), indent=4))

{
    "name": "Centerville Model Tornado",
    "description": "Centerville mean width tornado hazard",
    "tornadoType": "model",
    "tornadoModel": "MeanWidthTornado",
    "tornadoParameters": {
        "efRating": "EF5",
        "startLatitude": "35.218",
        "startLongitude": "-97.510",
        "randomSeed": "1234",
        "endLatitude": [
            35.246
        ],
        "endLongitude": [
            -97.438
        ],
        "windSpeedMethod": "1",
        "numSimulations": "1"
    }
}


In [5]:
model_response = hazardsvc.create_tornado_scenario(tornado_model_json)

In [6]:
print(json.dumps(model_response, indent=4))

{
    "tornadoType": "model",
    "id": "61003ecf1f652f176aeb5b23",
    "name": "Centerville Model Tornado",
    "description": "Centerville mean width tornado hazard",
    "creator": "mondrejc",
    "spaces": [
        "mondrejc"
    ],
    "date": "2021-07-27T17:13:50+0000",
    "tornadoModel": "MeanWidthTornado",
    "tornadoParameters": {
        "efRating": "EF5",
        "maxWindSpeed": 250.0,
        "startLatitude": 35.218,
        "startLongitude": -97.51,
        "randomSeed": 1234,
        "windSpeedMethod": 1,
        "numSimulations": 1,
        "endLatitude": [
            35.246
        ],
        "endLongitude": [
            -97.438
        ]
    },
    "tornadoWidth": [
        815.9980909090909
    ],
    "efBoxes": [
        {
            "efBoxWidths": [
                0.001002256638187928,
                0.0017328393158413994,
                0.0022321320000668874,
                0.0027387672237662794,
                0.003205018480359198,
                0.003

## Create Dataset Based Tornado

**Inputs:** Json representing a the dataset based tornado; Provide dataset representing the tornado path in shapefile format

In [7]:
with open("files/tornado-dataset.json", 'r') as file:
    tornado_dataset_json = file.read()
    print(json.dumps(json.loads(tornado_dataset_json), indent=4))

{
    "name": "Joplin Dataset Tornado",
    "description": "Joplin tornado hazard with shapefile",
    "tornadoType": "dataset"
}


In [8]:
file_paths = ["files/joplin_path_wgs84.shp", "files/joplin_path_wgs84.shx", "files/joplin_path_wgs84.dbf", 
              "files/joplin_path_wgs84.prj", "files/joplin_path_wgs84.qpj"] 

In [9]:
dataset_response = hazardsvc.create_tornado_scenario(tornado_dataset_json, file_paths)
print(json.dumps(dataset_response, indent=4))

{
    "tornadoType": "dataset",
    "id": "61003ed1720d7a6076acd3b4",
    "name": "Joplin Dataset Tornado",
    "description": "Joplin tornado hazard with shapefile",
    "creator": "mondrejc",
    "spaces": [
        "mondrejc"
    ],
    "date": "2021-07-27T17:13:52+0000",
    "datasetId": "61003ed060b3f41243022450"
}


## Get Tornado Values

In [10]:
tornado_model_id = model_response['id']
points = [
    {
        "demands": ["wind"],
        "units": ["mph"],
        "loc": "35.215, -97.521"
    },
    {
        "demands": ["wind"],
        "units": ["mph"],
        "loc": "35.227, -97.471"
    }
]
tornado_model_vals = hazardsvc.post_tornado_hazard_values(tornado_model_id, points)
print(tornado_model_vals)

[{'hazardValues': [0.0], 'demands': ['Wind'], 'units': ['mph'], 'loc': '35.215, -97.521'}, {'hazardValues': [0.0], 'demands': ['Wind'], 'units': ['mph'], 'loc': '35.227, -97.471'}]


In [11]:
tornado_dataset_id = dataset_response['id']
points = [
    {
        "demands": ["wind"],
        "units": ["mph"],
        "loc": "37.066, -94.502"
    },
    {
        "demands": ["wind"],
        "units": ["mph"],
        "loc": "37.032, -94.348"
    }
]
tornado_dataset_vals = hazardsvc.post_tornado_hazard_values(tornado_dataset_id, points)
print(tornado_dataset_vals)

[{'hazardValues': [226.26049288304796], 'demands': ['Wind'], 'units': ['mph'], 'loc': '37.066, -94.502'}, {'hazardValues': [101.9010355084985], 'demands': ['Wind'], 'units': ['mph'], 'loc': '37.032, -94.348'}]
